# Churn Analysis

In [1]:
# Import Packages
import pandas as pd
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt

import math
from datetime import datetime, timedelta

#import scikit-learn
#import jupyterlab
#import seaborn
#import xgboost

# Set Display Preferences
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

### Load & manipulate data

In [2]:
### UK Based ecom retailer sales from Jan 12, 2009 to Sep 12, 2011.
### Mainly sells unique all-occasion gift-ware.
df1 = pd.read_excel("data/online_retail_II.xlsx", sheet_name='Year 2009-2010', header=0)
df2 = pd.read_excel("data/online_retail_II.xlsx", sheet_name='Year 2010-2011', header=0)

### Note: Both excel sheets contain 2010 sales. Duplicates must be removed before concatenating.
# There is 1 week of overlap in the datasets
print(df1['InvoiceDate'].max())
print(df2['InvoiceDate'].min())

# Remove duplicate entries 
df1 = df1[~df1["Invoice"].isin(df2['Invoice'].unique())]
# Note: achives identical result as -> df1[df1['InvoiceDate']<df2['InvoiceDate'].min()] 

2010-12-09 20:01:00
2010-12-01 08:26:00


### Define new factors

In [ ]:
# Merge the dataframes from each year
df = pd.concat([df1, df2], axis=0, ignore_index=True)

### Create a Invoice Amount column
df['Invoice Amount'] = df['Price'] * df['Quantity']

### Sales are overwhelmingly from the UK.
uk_pct = len(df[df['Country']=='United Kingdom']) / len(df)
print(f'Percent of entries sold in the UK: {uk_pct*100:.2f}%')

### Create a binary variable indicating UK vs. Global.
df['UK_Ind'] = np.where(df['Country']=='United Kingdom', 1, 0)

### Remove time component from Invoice Date
df['InvoiceDate'] = df['InvoiceDate'].dt.date

df_orig = df.copy()

Percent of entries sold in the UK: 91.88%


### Remove appropriate invoice entries

In [9]:
### Null Customer IDs (required to determine churn). 
### Reason for missing Customer ID is unknown. For simplicity, these entries will be excluded from the analysis. This likely creates a source of bias.
### Recomendation is to update the ordering platform to require purchasers to create an account, generating a unique Customer ID.
null_id_pct = sum(df['Customer ID'].isnull()) / len(df)
print(f'Percent of entries that are missing Customer ID: {null_id_pct*100:.2f}%')


### Some items have a price of $0. These entries will not count towards orders.
### Primarily consisting of car flags, stickers, shipping materials & invoice notes.
non_items = len(df[df['Price']==0]) / len(df)
print(f'Percent of entries that had a price of $0: {non_items*100:.2f}%')


### Examining the Stock Codes, entries starting with alphabetic characters are atypical invoice entries that do not represent merchandise sales.
### Ex. Test products, adjustments, postage, carriage, etc.
df["StockCode"] = df["StockCode"].astype(str)
stockcode = (
    df.groupby(["StockCode", "Description"])["Quantity"]
      .sum()
      .reset_index()        
      .sort_values("StockCode", ascending=False)
    )

stockcode_pct = sum(~df['StockCode'].str.match(r'^\d')) / len(df)
print(f'Percent of entries that have invalid Stock Codes: {stockcode_pct*100:.2f}%')


### Remove entries is no Customer ID or Price = $0
df = df[~df['Customer ID'].isnull()]
df = df[df['Price']!=0]
### Remove entries for Stock Codes beginning with alphabetic characters
df = df[df['StockCode'].str.match(r'^\d')]

Percent of entries that are missing Customer ID: 22.52%
Percent of entries that had a price of $0: 0.58%
Percent of entries that have invalid Stock Codes: 0.57%


### Aggregate data to Customer ID granualrity

In [11]:
def aggregate_by_cx(df):
    """
    Aggregate transaction data & group it by Customer ID
    """
    df = df.reset_index()
    df_aggregated = df.groupby(['Customer ID', 'UK_Ind'], as_index=False).agg(
        # First & last txn in the measurement period
        first_txn = ('InvoiceDate', 'min'),
        last_txn = ('InvoiceDate', 'max'),
        # Transaction Count
        total_txns = ('Invoice', 'nunique'),
        sale_txns = ('Invoice', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].nunique()),
        return_txns = ('Invoice', lambda x: x[df.loc[x.index, 'Quantity'] < 0].nunique()),
        # Sales amount ($)
        total_amt = ('Invoice Amount', 'sum'),
        sale_amt = ('Invoice Amount', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].sum()),
        return_amt = ('Invoice Amount', lambda x: x[df.loc[x.index, 'Quantity'] < 0].sum()),
        # Sale quantity
        total_qty = ('Quantity', 'sum'),
        sale_qty = ('Quantity', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].sum()),
        return_qty = ('Quantity', lambda x: x[df.loc[x.index, 'Quantity'] < 0].sum()),
        # Variaty of SKUs
        total_skus = ('StockCode', 'nunique'),
        sale_skus = ('StockCode', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].nunique())
        )

    # Returns Percent
    df_aggregated['return_pct'] = df_aggregated['return_amt'] / df_aggregated['sale_amt']
    # Units per Transaction
    df_aggregated['upt'] = df_aggregated['total_qty'] / df_aggregated['total_txns']
    # Average Unit Retail
    df_aggregated['aur'] = df_aggregated['total_amt'] / df_aggregated['total_qty']
    # Average Order Value
    df_aggregated['aov'] = df_aggregated['total_amt'] / df_aggregated['total_txns']
    
    return df_aggregated

In [53]:
### Check to confirm there are no Customer IDs found in multiple markets/countries
df_aggregated = aggregate_by_cx(df)
cx_market = df_aggregated.groupby("Customer ID")["UK_Ind"].count()
multi_market_cx = sum(cx_market>1)
print('Number of customers in multiple markets:', multi_market_cx)

Number of customers in multiple markets: 0


In [ ]:
### Churn will be definited by if a customer purchases in the next 180 days.
### The past 180 days will be used to train a model that predicts likelihood to churn. This will be called the observation window.
### The dataset spans 2 years, so this approach evaluates churn in a recurant sliding window approach.
observation_days = 180
observation_periods = (df['InvoiceDate'].max() - df['InvoiceDate'].min()).days / observation_days
observation_periods = int(math.floor(observation_periods))
print('Number of', observation_days, 'day periods:', observation_periods, '\n')

# Slice data by observation period starting from most recent & working backward by increments equal to the duration of the observation period
end = df['InvoiceDate'].max()
df_churn_ind = pd.DataFrame()
for i in range(observation_periods):
    # Get start & end date for the given observation period.
    period_end = end
    period_start = period_end - timedelta(observation_days)
    # Create a temporary dataframe for the observation period. Aggregate by Customer ID & compute customer satistics.
    df_i = df[df['InvoiceDate'].between(period_start,period_end)]
    df_i = aggregate_by_cx(df_i)
    
    # Tag if customer churns in next period
    if i==0:
        churn_ind = np.nan
    else:
        churn_ind = df_i['Customer ID'].isin(df_next_period['Customer ID']).map({True: 'no', False: 'yes'})
    # Define new columns for Churn Indicator & Observation Period
    df_i.insert(loc=0, column='Churn Next Period', value=churn_ind)
    df_i.insert(loc=0, column='Period', value=observation_periods-i)
    df_i.insert(loc=0, column='Observation End', value=period_end)
    df_i.insert(loc=0, column='Observation Start', value=period_start)

    # Concatenate each period's customer data into a single dataframe
    df_churn_ind = pd.concat([df_churn_ind, df_i], ignore_index=True)

    # Updated end date for next period
    df_next_period = df_i.copy()
    end = period_start - timedelta(days=1)

df_churn_ind = df_churn_ind.sort_values(by=['Customer ID', 'Observation Start'],ignore_index=True)


Number of 90 day periods: 8 



,Observation Start,Observation End,Period,Churn Next Period,Customer ID,UK_Ind,first_txn,last_txn,total_txns,sale_txns,return_txns,total_amt,sale_amt,return_amt,total_qty,sale_qty,return_qty,total_skus,sale_skus,return_pct,upt,aur,aov
0,2009-12-12,2010-03-12,1,yes,12346.00,1,2010-03-02,2010-03-02,1,1,0,27.05,27.05,0.00,5,5,0,5,5,0.00,5.00,5.41,27.05
1,2010-06-12,2010-09-10,3,yes,12346.00,1,2010-06-28,2010-06-28,1,1,0,142.31,142.31,0.00,19,19,0,19,19,0.00,19.00,7.49,142.31
2,2010-12-11,2011-03-11,5,yes,12346.00,1,2011-01-18,2011-01-18,2,1,1,0.00,77183.60,-77183.60,0,74215,-74215,1,1,-1.00,0.00,NaN,0.00
3,2010-09-11,2010-12-10,4,no,12347.00,0,2010-10-31,2010-12-07,2,2,0,1323.32,1323.32,0.00,828,828,0,70,70,0.00,414.00,1.60,661.66
4,2010-12-11,2011-03-11,5,no,12347.00,0,2011-01-26,2011-01-26,1,1,0,475.39,475.39,0.00,315,315,0,29,29,0.00,315.00,1.51,475.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17121,2010-06-12,2010-09-10,3,yes,18286.00,1,2010-06-24,2010-08-20,2,1,1,725.48,833.48,-108.00,478,494,-16,53,50,-0.13,239.00,1.52,362.74
17122,2010-03-13,2010-06-11,2,yes,18287.00,1,2010-05-17,2010-05-17,1,1,0,1071.61,1071.61,0.00,732,732,0,54,54,0.00,732.00,1.46,1071.61
17123,2010-09-11,2010-12-10,4,yes,18287.00,1,2010-09-21,2010-11-22,3,3,0,1224.10,1224.10,0.00,694,694,0,28,28,0.00,231.33,1.76,408.03
17124,2011-03-12,2011-06-10,6,yes,18287.00,1,2011-05-22,2011-05-22,1,1,0,765.28,765.28,0.00,488,488,0,27,27,0.00,488.00,1.57,765.28


In [ ]:
#35% churn 180 days

#42% churn 90 days